<a href="https://colab.research.google.com/github/nautiyal-sarthak/tesla-dash/blob/main/aura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thinks that i want to capture : 

1. Latest Odometer reading

**Summary**
1. Distance traveled per day/Month
2. Trip Efficincy per Day/Month
3. Charign efficincy per day/Month
4. cost to travel 1 km per day/month compared to ice
5. Phantom drain per day/Month

**Details of the charge session**
1. Charge efficincy
2. rate of charge across time 
3. KMs traveled
4. Phantom drain

**Details of trips**
6. elevation 
7. Temprature 
8. efficincy
9. money saved

**Capture Data**

In [ ]:
from google.colab import auth
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from google.colab import drive
import os



auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
#Constants
total_kwh=50
max_range=400
kwh_per_km = total_kwh/max_range
super_charge_rate_per_kwh = 0.24
level2_rate_per_hour = 1
level1_rate_per_hour = 0.073

fule_price_per_l = 1.20
ice_fule_per_100km = 7 
cost_per_km_ice = ice_fule_per_100km * fule_price_per_l/100

In [ ]:
trip_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/16BraPT4BMTZZu_8q7DxqlZYI1ITXjnb3MUR5GEB5Qo4/edit#gid=0')
charge_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1Lq519W-aIu9kERUwDgY1Z6W0xIjChPk7WSf58wM5ZWw/edit#gid=0')
phantom_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1rvgmptBj0Yec7m6-E8t1V-Mdnv3HI2Fm0Y0qnuwjsnY/edit#gid=0')
master_wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1ShoBS_CPHui4tgtHuYM9KOR3diegqoUuEOR9tUag-eg/edit#gid=863619902')


trip_sheet = trip_wb.sheet1
charge_sheet = charge_wb.sheet1
phantom_sheet = phantom_wb.sheet1
#master_sheet = master_wb.worksheet('Aura_201224')


**Charge Details**

In [ ]:
charge_data = charge_sheet.get_all_values()
charge_df = pd.DataFrame(charge_data,columns=charge_data[0])
charge_df.drop(0,inplace=True)

charge_df.set_index("session_id",inplace=True)
charge_df.index = charge_df.index.astype(int,copy=False)
charge_df = charge_df[charge_df["session_end_time"] != ""]

#inforsing schema
charge_df = charge_df.astype(dtype= {
"session_date":"int64",
"session_start_time":"long",
"session_start_odometer":"float",
"session_start_longitude":"float",
"session_start_latitude":"float",
"session_start_ideal_battery_range":"float",
"session_start_energy_added":"float",
"session_start_battery_range":"float",
"session_start_miles_added_ideal":"float",
"session_start_miles_added_rated":"float",
"session_start_est_battery_range":"float",
"session_start_usable_battery_level":"int64",
"session_start_battery_level":"int64",
"session_end_odometer":"float",
"session_end_longitude":"float",
"session_end_latitude":"float",
"session_end_ideal_battery_range":"float",
"session_end_energy_added":"float",
"session_end_battery_range":"float",
"session_end_miles_added_ideal":"float",
"session_end_miles_added_rated":"float",
"session_end_est_battery_range":"float",
"session_end_usable_battery_level":"int64",
"session_end_battery_level":"int64",
"session_end_time":"long",
"charge_voltage":"string",
"is_free":"string"
})


#adding important cols
charge_df["session_start_time"] = pd.to_datetime(charge_df['session_start_time'], unit='ms').dt.tz_localize("utc").dt.tz_convert('America/Montreal')
charge_df["session_end_time"] = pd.to_datetime(charge_df['session_end_time'], unit='ms').dt.tz_localize("utc").dt.tz_convert('America/Montreal')

#charge level
def charge_lvl(row):
  charge_voltage = int(row['charge_voltage'])

  if charge_voltage <= 150:
      chr_typ = "Level 1"
  elif charge_voltage > 150 and charge_voltage < 450 :
      chr_typ = "Level 2"
  elif charge_voltage >= 450 :
      chr_typ = "Level 3"
  
  return chr_typ

charge_df["charge_level"] = charge_df.apply(charge_lvl, axis=1)

#1. Charge durtion
charge_df["charge_duration"] = (charge_df["session_end_time"] - charge_df["session_start_time"]).dt.total_seconds()

charge_df["session_start_date"] = charge_df["session_start_time"].dt.date


#2. KWh added
charge_df["kwh_added"] = charge_df["session_end_energy_added"] - charge_df["session_start_energy_added"]

#3. Km added
charge_df["km_added"] = (charge_df["session_end_miles_added_ideal"] - charge_df["session_start_miles_added_ideal"]) * 1.60934

#4. Cost
def calCost(row):
  level = row['charge_level']
  is_free = row['is_free']
  duration = row['charge_duration']/(60*60)
  kwh_added = row['kwh_added']
  cost = 0

  if is_free != "TRUE" :
    if level == "Level 3" :
        cost = kwh_added * super_charge_rate_per_kwh
    elif level == "Level 2" :
        cost = duration * level2_rate_per_hour
    elif level == "Level 1" :
        cost = duration * level1_rate_per_hour
  
  return cost


charge_df['cost'] = charge_df.apply(calCost, axis=1)

#5. distance traveled
charge_df["next_session_start"] = charge_df["session_start_odometer"].shift(-1)
charge_df["Actual_Distance_km"] = (charge_df["next_session_start"] - charge_df["session_end_odometer"] )  * 1.60934

#6. Cost per KM traveled
charge_df["cost/km"] = charge_df['cost']/charge_df["Actual_Distance_km"]

In [ ]:
charge_df_cal = charge_df[["session_date","session_start_date","session_start_time","session_end_time","charge_duration","session_start_battery_level","session_end_battery_level","kwh_added","km_added","cost","Actual_Distance_km","cost/km","charge_level"]]

**Trip Details**

In [ ]:
trip_data = trip_sheet.get_all_values()
trip_df = pd.DataFrame(trip_data,columns=trip_data[0])
trip_df.drop(0,inplace=True)
trip_df.set_index("trip_id",inplace=True)
trip_df = trip_df[trip_df["trip_end_time"] != ""]


#inforsing schema
trip_df = trip_df.astype(dtype= {"trip_date":"int64",
"charge_session_id":"int64",
"trip_start_time":"long",
"trip_start_odometer":"float",
"trip_start_longitude":"float",
"trip_start_latitude":"float",
"trip_start_ideal_battery_range":"float",
"trip_start_energy_added":"float",
"trip_start_battery_range":"float",
"trip_start_miles_added_ideal":"float",
"trip_start_miles_added_rated":"float",
"trip_start_est_battery_range":"float",
"trip_start_usable_battery_level":"int64",
"trip_start_battery_level":"int64",
"trip_start_temp":"float",
"trip_end_odometer":"float",
"trip_end_longitude":"float",
"trip_end_latitude":"float",
"trip_end_ideal_battery_range":"float",
"trip_end_energy_added":"float",
"trip_end_battery_range":"float",
"trip_end_miles_added_ideal":"float",
"trip_end_miles_added_rated":"float",
"trip_end_est_battery_range":"float",
"trip_end_usable_battery_level":"int64",
"trip_end_battery_level":"int64",
"trip_end_temp":"float",
"trip_end_time":"long"})

In [ ]:
#TRIP
#creating impotant cols
#1. Convert epoc time to local time
trip_df["trip_start_time"] = pd.to_datetime(trip_df['trip_start_time'], unit='ms').dt.tz_localize("utc").dt.tz_convert('America/Montreal')
trip_df["trip_end_time"] = pd.to_datetime(trip_df['trip_end_time'], unit='ms').dt.tz_localize("utc").dt.tz_convert('America/Montreal')
trip_df["trip_start_date"] = trip_df["trip_start_time"].dt.date

#2. Diffrence of odometer reading - distance traveled
trip_df["distance_traveled"] = (trip_df["trip_end_odometer"] - trip_df["trip_start_odometer"]) * 1.60934

#3. diffence of ideal_battery_range
trip_df["ideal_battery_range_used"] = (trip_df["trip_start_ideal_battery_range"]  - trip_df["trip_end_ideal_battery_range"]) * 1.60934

#7. diffrence of battery_level
trip_df["battery_%_used"] = trip_df["trip_start_battery_level"] - trip_df["trip_end_battery_level"] 

#8. avg of temp
trip_df["avg_temp"] = (trip_df["trip_end_temp"] + trip_df["trip_start_temp"])/2 

#9. duration of the trip
trip_df["duration"] = (trip_df["trip_end_time"] - trip_df["trip_start_time"]).dt.total_seconds()

#10. Trip Efficiency = distance traveled/diffence of ideal_battery_range 
trip_df["efficiency"] = trip_df["distance_traveled"] / trip_df["ideal_battery_range_used"] * 100

#11. max range = trip_end_ideal_battery_range/(trip_end_battery_level/100)
trip_df["max_range"] = (trip_df["trip_end_ideal_battery_range"] / (trip_df["trip_end_battery_level"] / 100)) * 1.60934

#12. max kwh =  max range * kwh_per_km
trip_df["max_kwh"] = trip_df["max_range"] * kwh_per_km

#11. Estimated kwh used =  (max kwh/max range)*distance traveled
trip_df["Estimated_kwh_used"] = (trip_df["battery_%_used"]/100) * trip_df["max_kwh"]

#13. wh/km =  max kwh/max range
trip_df["wh/km"] = trip_df["Estimated_kwh_used"] / trip_df["distance_traveled"] *1000

#14 trip cost
trip_df = trip_df.merge(charge_df_cal,left_on='charge_session_id', right_index=True)
trip_df["cost_ev"] = trip_df["distance_traveled"] * trip_df["cost/km"]

#15 approx cost for ice
trip_df["cost_ice"] = trip_df["distance_traveled"] * cost_per_km_ice

#16 approx cost for ice
trip_df["money_saved"] =  trip_df["cost_ice"] - trip_df["cost_ev"] 

#17 approv distance 
#10. Trip Efficiency = distance traveled/diffence of ideal_battery_range 
trip_df["real_dis"] = (trip_df["efficiency"]/100) * trip_df["max_range"]

In [ ]:
trip_df_cal = trip_df[["charge_session_id","trip_start_date","trip_start_time","trip_end_time","duration","avg_temp","distance_traveled","ideal_battery_range_used","efficiency","Estimated_kwh_used","wh/km","max_range","max_kwh","real_dis","cost_ev","cost_ice","money_saved"]]
trip_df_cal.fillna(0,inplace=True)
trip_df_cal = trip_df_cal[trip_df_cal["ideal_battery_range_used"] > 0]

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


**Phantom Discharge**

In [ ]:
phantom_data = phantom_sheet.get_all_values()
phantom_df = pd.DataFrame(phantom_data,columns=phantom_data[0])
phantom_df.drop(0,inplace=True)
phantom_df.set_index("park_id",inplace=True)
phantom_df = phantom_df[phantom_df["park_end_time"] != ""]

#inforsing schema
phantom_df = phantom_df.astype(dtype= {
"park_date":"int64",
"charge_session_id":"int64",
"park_start_time":"long",
"park_start_odometer":"float",
"park_start_longitude":"float",
"park_start_latitude":"float",
"park_start_ideal_battery_range":"float",
"park_start_energy_added":"float",
"park_start_battery_range":"float",
"park_start_miles_added_ideal":"float",
"park_start_miles_added_rated":"float",
"park_start_est_battery_range":"float",
"park_start_usable_battery_level":"int64",
"park_start_battery_level":"int64",
"park_start_temp":"float",
"park_sentry_mode":"string",
"park_end_odometer":"float",
"park_end_longitude":"float",
"park_end_latitude":"float",
"park_end_ideal_battery_range":"float",
"park_end_energy_added":"float",
"park_end_battery_range":"float",
"park_end_miles_added_ideal":"float",
"park_end_miles_added_rated":"float",
"park_end_est_battery_range":"float",
"park_end_usable_battery_level":"int64",
"park_end_battery_level":"int64",
"park_end_temp":"float",
"park_end_time":"long"
})

In [ ]:
#adding important cols
phantom_df["park_start_time"] = pd.to_datetime(phantom_df['park_start_time'], unit='ms').dt.tz_localize("utc").dt.tz_convert('America/Montreal')
phantom_df["park_end_time"] = pd.to_datetime(phantom_df['park_end_time'], unit='ms').dt.tz_localize("utc").dt.tz_convert('America/Montreal')
phantom_df["park_start_date"] = phantom_df["park_start_time"].dt.date

#1. park durtion
phantom_df["duration"] = (phantom_df["park_end_time"] - phantom_df["park_start_time"]).dt.total_seconds()

#2. diffrence of battery_level
phantom_df["battery_%_lost"] = phantom_df["park_start_battery_level"] - phantom_df["park_end_battery_level"]

#3. max range = trip_end_ideal_battery_range/(trip_end_battery_level/100)
phantom_df["max_range"] = (phantom_df["park_end_ideal_battery_range"] / (phantom_df["park_end_battery_level"] / 100)) * 1.60934

#4. max kwh =  max range * kwh_per_km
phantom_df["max_kwh"] = phantom_df["max_range"] * kwh_per_km

#5. Estimated kwh lost =  (max kwh/max range)*distance traveled
phantom_df["Estimated_kwh_lost"] = (phantom_df["battery_%_lost"]/100) * phantom_df["max_kwh"]

#6. avg of temp
phantom_df["avg_temp"] = (phantom_df["park_end_temp"] + phantom_df["park_start_temp"])/2 

#7. rate of energy(wh) loss per hour
phantom_df["wh_loss_rate_per_hr"] = (phantom_df["Estimated_kwh_lost"]*1000)/(phantom_df["duration"]/(60*60))


In [ ]:
phantom_df_cal = phantom_df[["park_date","park_start_date","charge_session_id","park_sentry_mode","park_start_time","park_end_time","duration","avg_temp","battery_%_lost","Estimated_kwh_lost","wh_loss_rate_per_hr"]]

In [ ]:
phantom_df[["park_start_battery_level","park_start_battery_level"]]

,park_start_battery_level,park_start_battery_level
park_id,,
1,26,26
2,24,24
3,23,23
4,74,74
5,74,74
...,...,...
591,39,39
592,38,38
593,36,36


**+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

In [ ]:
#Writting to Gdrive
if(os.path.exists("/gdrive/My Drive/aura_data/charge.csv")) :
    os.remove("/gdrive/My Drive/aura_data/charge.csv")

if(os.path.exists("/gdrive/My Drive/aura_data/phantom.csv")) :
    os.remove("/gdrive/My Drive/aura_data/phantom.csv")

if(os.path.exists("/gdrive/My Drive/aura_data/trip.csv")):
    os.remove("/gdrive/My Drive/aura_data/trip.csv")


phantom_df_cal.to_csv("/gdrive/My Drive/aura_data/phantom.csv")
charge_df_cal.to_csv("/gdrive/My Drive/aura_data/charge.csv")
trip_df_cal.to_csv("/gdrive/My Drive/aura_data/trip.csv")


In [ ]:
charge_df

,session_date,session_start_time,session_start_odometer,session_start_longitude,session_start_latitude,session_start_ideal_battery_range,session_start_energy_added,session_start_battery_range,session_start_miles_added_ideal,session_start_miles_added_rated,session_start_est_battery_range,session_start_usable_battery_level,session_start_battery_level,session_end_odometer,session_end_longitude,session_end_latitude,session_end_ideal_battery_range,session_end_energy_added,session_end_battery_range,session_end_miles_added_ideal,session_end_miles_added_rated,session_end_est_battery_range,session_end_usable_battery_level,session_end_battery_level,session_end_time,charge_voltage,is_free,charge_level,charge_duration,session_start_date,kwh_added,km_added,cost,next_session_start,Actual_Distance_km,cost/km
session_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,201225,2020-12-25 21:31:04.699000-05:00,562.690166,-73.683953,45.512136,58.73,29.94,58.73,146.0,146.0,50.69,23,24,562.690166,-73.683953,45.512136,58.73,29.94,58.73,146.0,146.0,50.69,23,24,2020-12-25 21:31:04.699000-05:00,240,FALSE,Level 2,0.000,2020-12-25,0.00,0.00000,0.000000,563.259342,0.915998,0.000000
2,201226,2020-12-26 12:45:01.033000-05:00,563.259342,-73.690199,45.516116,41.88,0.00,41.88,0.0,0.0,35.08,16,18,563.259342,-73.690199,45.516116,59.24,3.56,59.24,17.5,17.5,49.63,23,24,2020-12-26 13:27:01.541000-05:00,240,FALSE,Level 2,2520.508,2020-12-26,3.56,28.16345,0.700141,564.386510,1.813997,0.385966
3,201226,2020-12-26 13:42:01.402000-05:00,564.386510,-73.683118,45.512818,55.67,0.10,55.67,0.5,0.5,45.21,21,23,564.386510,-73.683118,45.512818,205.31,30.78,205.31,150.0,150.0,166.72,79,79,2020-12-26 18:24:01.604000-05:00,240,FALSE,Level 2,16920.202,2020-12-26,30.68,240.59633,4.700056,624.417805,96.610764,0.048649
4,201228,2020-12-28 11:31:22.232000-05:00,624.417805,-73.683099,45.512822,54.65,0.00,54.65,0.0,0.0,40.06,21,23,624.417805,-73.683099,45.512822,192.03,28.16,192.03,137.5,137.5,140.77,74,74,2020-12-28 15:48:22.232000-05:00,240,FALSE,Level 2,15420.000,2020-12-28,28.16,221.28425,4.283333,727.319986,165.604596,0.025865
5,201229,2020-12-29 12:08:22.535000-05:00,727.319986,-73.683137,45.512822,35.75,0.00,35.75,0.0,0.0,29.80,14,15,727.319986,-73.683137,45.512822,94.99,12.14,94.99,59.0,59.0,79.18,36,37,2020-12-29 13:54:22.464000-05:00,240,FALSE,Level 2,6359.929,2020-12-29,12.14,94.95106,1.766647,744.664942,27.913931,0.063289
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,210407,2021-04-07 17:40:49.539000-04:00,3376.492073,-73.682650,45.507858,70.99,0.00,70.99,0.0,0.0,72.73,27,27,3376.492073,-73.682650,45.507858,91.93,4.29,91.93,21.0,21.0,94.18,35,35,2021-04-07 18:27:49.580000-04:00,201,,Level 2,2820.041,2021-04-07,4.29,33.79614,0.783345,3400.036450,37.890908,0.020674
70,210408,2021-04-08 10:46:49.568000-04:00,3400.036450,-73.683062,45.512848,48.51,0.00,48.51,0.0,0.0,48.72,19,20,3400.036450,-73.683062,45.512848,221.65,35.60,221.65,173.5,173.5,223.11,85,85,2021-04-08 15:55:49.383000-04:00,239,,Level 2,18539.815,2021-04-08,35.60,279.22049,5.149949,3444.434045,71.450826,0.072077
71,210413,2021-04-13 12:36:49.528000-04:00,3444.434045,-73.683154,45.512816,89.87,0.00,89.87,0.0,0.0,94.69,34,34,3444.434045,-73.683154,45.512816,169.02,16.23,169.02,79.0,79.0,177.62,65,65,2021-04-13 14:57:49.307000-04:00,240,,Level 2,8459.779,2021-04-13,16.23,127.13786,2.349939,3448.452452,6.466983,0.363375


In [ ]:
trip_df

,trip_date,charge_session_id,trip_start_time,trip_start_odometer,trip_start_longitude,trip_start_latitude,trip_start_ideal_battery_range,trip_start_energy_added,trip_start_battery_range,trip_start_miles_added_ideal,trip_start_miles_added_rated,trip_start_est_battery_range,trip_start_usable_battery_level,trip_start_battery_level,trip_start_temp,trip_end_odometer,trip_end_longitude,trip_end_latitude,trip_end_ideal_battery_range,trip_end_energy_added,trip_end_battery_range,trip_end_miles_added_ideal,trip_end_miles_added_rated,trip_end_est_battery_range,trip_end_usable_battery_level,trip_end_battery_level,trip_end_temp,trip_end_time,trip_start_date,distance_traveled,ideal_battery_range_used,battery_%_used,avg_temp,duration,efficiency,max_range,max_kwh,Estimated_kwh_used,wh/km,session_date,session_start_date,session_start_time,session_end_time,charge_duration,session_start_battery_level,session_end_battery_level,kwh_added,km_added,cost,Actual_Distance_km,cost/km,charge_level,cost_ev,cost_ice,money_saved,real_dis
trip_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,201227,3,2020-12-27 18:58:21.981000-05:00,596.911565,-73.685033,45.511020,101.12,30.78,101.12,150.0,150.0,60.97,39,40,-4.0,624.225180,-73.684179,45.511973,66.90,30.78,66.90,150.0,150.0,49.69,26,26,-3.5,2020-12-27 19:46:21.516000-05:00,2020-12-27,43.956893,55.071615,14,-3.75,2879.535,79.817694,414.095562,51.761945,7.246672,164.858610,201226,2020-12-26,2020-12-26 13:42:01.402000-05:00,2020-12-26 18:24:01.604000-05:00,16920.202,23,79,30.68,240.59633,4.700056,96.610764,0.048649,Level 2,2.138477,3.692379,1.553902,330.521530
2,201228,3,2020-12-28 10:00:32.092000-05:00,624.225801,-73.684167,45.511968,56.69,30.78,56.69,150.0,150.0,42.11,22,23,1.5,624.225801,-73.684167,45.511968,56.69,30.78,56.69,150.0,150.0,42.11,22,23,1.5,2020-12-28 10:00:32.092000-05:00,2020-12-28,0.000000,0.000000,0,1.50,0.000,NaN,396.667324,49.583416,0.000000,NaN,201226,2020-12-26,2020-12-26 13:42:01.402000-05:00,2020-12-26 18:24:01.604000-05:00,16920.202,23,79,30.68,240.59633,4.700056,96.610764,0.048649,Level 2,0.000000,0.000000,0.000000,NaN
3,201228,3,2020-12-28 11:27:22.458000-05:00,624.233258,-73.684270,45.511969,55.16,30.78,55.16,150.0,150.0,40.73,21,23,0.5,624.417805,-73.683099,45.512822,54.65,30.78,54.65,150.0,150.0,40.06,21,23,0.5,2020-12-28 11:30:22.063000-05:00,2020-12-28,0.296999,0.820763,0,0.50,179.605,36.185686,382.393178,47.799147,0.000000,0.000000,201226,2020-12-26,2020-12-26 13:42:01.402000-05:00,2020-12-26 18:24:01.604000-05:00,16920.202,23,79,30.68,240.59633,4.700056,96.610764,0.048649,Level 2,0.014449,0.024948,0.010499,138.371596
4,201228,4,2020-12-28 15:49:22.665000-05:00,624.478699,-73.683921,45.512240,192.03,28.16,192.03,137.5,137.5,140.14,74,74,4.0,624.485534,-73.683922,45.512270,192.03,28.16,192.03,137.5,137.5,140.14,74,74,4.0,2020-12-28 15:51:22.306000-05:00,2020-12-28,0.011000,0.000000,0,4.00,119.641,inf,417.623730,52.202966,0.000000,0.000000,201228,2020-12-28,2020-12-28 11:31:22.232000-05:00,2020-12-28 15:48:22.232000-05:00,15420.000,23,74,28.16,221.28425,4.283333,165.604596,0.025865,Level 2,0.000285,0.000924,0.000639,inf
5,201228,4,2020-12-28 15:53:22.287000-05:00,624.535865,-73.684601,45.511704,192.03,28.16,192.03,137.5,137.5,140.14,74,74,4.0,677.925323,-73.683946,45.512152,121.55,28.16,121.55,137.5,137.5,117.30,47,47,3.0,2020-12-28 18:21:22.148000-05:00,2020-12-28,85.921790,113.426283,27,3.50,8879.861,75.751217,416.202717,52.025340,14.046842,163.484043,201228,2020-12-28,2020-12-28 11:31:22.232000-05:00,2020-12-28 15:48:22.232000-05:00,15420.000,23,74,28.16,221.28425,4.283333,165.604596,0.025865,Level 2,2.222352,7.217430,4.995079,315.278625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,210415,72,2021-04-15 16:57:49.144000-04:00,3474.302117,-73.685729,45.510949,103.13,1.78,103.13,8.5,8.

In [ ]:
charge_df_cal

,session_date,session_start_date,session_start_time,session_end_time,charge_duration,session_start_battery_level,session_end_battery_level,kwh_added,km_added,cost,Actual_Distance_km,cost/km,charge_level
session_id,,,,,,,,,,,,,
1,201225,2020-12-25,2020-12-25 21:31:04.699000-05:00,2020-12-25 21:31:04.699000-05:00,0.000,24,24,0.00,0.00000,0.000000,0.915998,0.000000,Level 2
2,201226,2020-12-26,2020-12-26 12:45:01.033000-05:00,2020-12-26 13:27:01.541000-05:00,2520.508,18,24,3.56,28.16345,0.700141,1.813997,0.385966,Level 2
3,201226,2020-12-26,2020-12-26 13:42:01.402000-05:00,2020-12-26 18:24:01.604000-05:00,16920.202,23,79,30.68,240.59633,4.700056,96.610764,0.048649,Level 2
4,201228,2020-12-28,2020-12-28 11:31:22.232000-05:00,2020-12-28 15:48:22.232000-05:00,15420.000,23,74,28.16,221.28425,4.283333,165.604596,0.025865,Level 2
5,201229,2020-12-29,2020-12-29 12:08:22.535000-05:00,2020-12-29 13:54:22.464000-05:00,6359.929,15,37,12.14,94.95106,1.766647,27.913931,0.063289,Level 2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,210407,2021-04-07,2021-04-07 17:40:49.539000-04:00,2021-04-07 18:27:49.580000-04:00,2820.041,27,35,4.29,33.79614,0.783345,37.890908,0.020674,Level 2
70,210408,2021-04-08,2021-04-08 10:46:49.568000-04:00,2021-04-08 15:55:49.383000-04:00,18539.815,20,85,35.60,279.22049,5.149949,71.450826,0.072077,Level 2
71,210413,2021-04-13,2021-04-13 12:36:49.528000-04:00,2021-04-13 14:57:49.307000-04:00,8459.779,34,65,16.23,127.13786,2.349939,6.466983,0.363375,Level 2


In [ ]:
trip_df_cal[trip_df_cal["real_dis"] > 100]

,charge_session_id,trip_start_date,trip_start_time,trip_end_time,duration,avg_temp,distance_traveled,ideal_battery_range_used,efficiency,Estimated_kwh_used,wh/km,max_range,max_kwh,real_dis,cost_ev,cost_ice,money_saved
trip_id,,,,,,,,,,,,,,,,,
1,3,2020-12-27,2020-12-27 18:58:21.981000-05:00,2020-12-27 19:46:21.516000-05:00,2879.535,-3.75,43.956893,55.071615,79.817694,7.246672,164.858610,414.095562,51.761945,330.521530,2.138477,3.692379,1.553902
3,3,2020-12-28,2020-12-28 11:27:22.458000-05:00,2020-12-28 11:30:22.063000-05:00,179.605,0.50,0.296999,0.820763,36.185686,0.000000,0.000000,382.393178,47.799147,138.371596,0.014449,0.024948,0.010499
5,4,2020-12-28,2020-12-28 15:53:22.287000-05:00,2020-12-28 18:21:22.148000-05:00,8879.861,3.50,85.921790,113.426283,75.751217,14.046842,163.484043,416.202717,52.025340,315.278625,2.222352,7.217430,4.995079
6,4,2020-12-29,2020-12-29 10:42:22.719000-05:00,2020-12-29 11:35:22.254000-05:00,3179.535,-6.75,46.415887,64.100012,72.411667,8.022238,172.833882,401.111902,50.138988,290.451815,1.200539,3.898935,2.698396
7,4,2020-12-29,2020-12-29 11:38:22.782000-05:00,2020-12-29 12:09:22.818000-05:00,1860.036,-6.50,32.906920,41.923307,78.493140,4.862889,147.777094,389.031123,48.628890,305.362744,0.851132,2.764181,1.913050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,72,2021-04-15,2021-04-15 15:06:49.093000-04:00,2021-04-15 15:11:50.079000-04:00,300.986,9.75,0.988997,0.820763,120.497255,0.000000,0.000000,412.815336,51.601917,497.431148,0.004631,0.083076,0.078445
413,72,2021-04-15,2021-04-15 16:57:49.144000-04:00,2021-04-15 17:04:49.246000-04:00,420.102,8.50,1.685996,3.283054,51.354510,0.521436,309.275022,417.149181,52.143648,214.224917,0.007894,0.141624,0.133729
414,72,2021-04-15,2021-04-15 17:39:49.238000-04:00,2021-04-15 17:45:49.719000-04:00,360.481,9.00,1.159996,1.657620,69.979612,0.527006,454.316937,421.604729,52.700591,295.037352,0.005431,0.097440,0.092008
